### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{'_wandb': {'runtime': 6}},{},ic01zuzq
1,{'_wandb': {'runtime': 6}},{},36415u3j
2,{'_wandb': {'runtime': 6}},{},1ung9wf5
3,{'_wandb': {'runtime': 6}},{},3569cd71
4,{},"{'exp_name': 'gpt_video_cons_reg0001', 'overri...",1vye579l
...,...,...,...
1495,"{'negative_sim': 0.04744461342226714, 'pos_neg...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1496,"{'pos_neg_diff': 0.4670497511327267, 'positive...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1497,"{'negative_sim': 0.18186782717704772, 'pos_neg...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1498,"{'_runtime': 17708, '_timestamp': 1659279862, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'gpt_video_cons_nproj_npred'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_141464/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
173,"{'reward_ratio': 0.42362118105905294, 'min_gra...","{'env': {'game': 'Krull', 'seed': 42, 'type': ...",fd7muegh,Krull
174,"{'obs_loss': 0.00016332345888266109, 'reg_loss...","{'env': {'game': 'KungFuMaster', 'seed': 42, '...",1dgh0mj4,KungFuMaster
175,"{'epoch': 10, 'act_acc': 0.14320501765907617, ...","{'env': {'game': 'Pong', 'seed': 42, 'type': '...",34btc9mq,Pong
176,"{'rank_eps1': 87, 'pos_neg_diff': 0.5474798679...","{'env': {'game': 'MsPacman', 'seed': 42, 'type...",1vr94y0x,MsPacman
177,"{'rank_eps001': 84, 'max_grad_norm': 0.0578534...","{'env': {'game': 'Kangaroo', 'seed': 42, 'type...",25twa67l,Kangaroo
178,"{'rank_eps01': 156, 'max_grad_norm': 0.0672287...","{'env': {'game': 'Qbert', 'seed': 42, 'type': ...",2zrxkw5m,Qbert
179,"{'reward_f1': 0.7724089635854341, 'reward_rati...","{'env': {'game': 'RoadRunner', 'seed': 42, 'ty...",29c271u9,RoadRunner
180,"{'reward_ratio': 0.004550227511375569, 'mean_g...","{'env': {'game': 'PrivateEye', 'seed': 42, 'ty...",3rh01hdu,PrivateEye
199,"{'epoch': 10, 'act_acc': 0.18508811022450283, ...","{'env': {'game': 'UpNDown', 'seed': 42, 'type'...",36qrucor,UpNDown
200,"{'reward_ratio': 0.07085354267713385, 'min_gra...","{'env': {'game': 'Seaquest', 'seed': 42, 'type...",30z7i60i,Seaquest


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

Krull fd7muegh
KungFuMaster 1dgh0mj4
Pong 34btc9mq
MsPacman 1vr94y0x
Kangaroo 25twa67l
Qbert 2zrxkw5m
RoadRunner 29c271u9
PrivateEye 3rh01hdu
UpNDown 36qrucor
Seaquest 30z7i60i
ChopperCommand 2j8k40rh
CrazyClimber 2zdw3eku
Jamesbond bu0ujfz3
Freeway m2xhx8m0
Gopher 12eqvmfm
Frostbite 2nibvri8
Hero 2d80ud8e
DemonAttack 2wpl9149
Amidar 2w1fkdh7
Alien 1v0yz77o
BankHeist 26z60kz6
BattleZone 37ml8wn2
Asterix 3s28bwpz
Boxing fsmi39mq
Breakout 2zh4ib77
Assault oibxw3iq


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)